In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from lib.imports import *
from lib.video_processor import VideoProcessor
from lib.misc import *

In [4]:
interpolate_rate=5
eyes_int_rate=1
eye_model_size=256
input_source = 0 # "example_video/example_vid_original.mp4"
form="onnx"


eye_batch_rate=(interpolate_rate//(eyes_int_rate+1))+1
convert_pt_to_onnx(model_path=f'models/eyes_{eye_model_size}_auto.pt', imgsz=eye_model_size, batch=eye_batch_rate, form=form)

processor = VideoProcessor(eye_model_size=eye_model_size, form=form)
out = processor.start(video_path=input_source, interpolate_rate=interpolate_rate, eyes_int_rate=eyes_int_rate, eye_batch_rate=eye_batch_rate, return_instead=False, disable_stats=False);

Ultralytics YOLOv8.2.9 🚀 Python-3.10.12 torch-2.2.1+cu121 CPU (11th Gen Intel Core(TM) i7-1165G7 2.80GHz)
WARNING ⚠️ half=True only compatible with GPU export, i.e. use device=0
YOLOv8n-pose summary (fused): 187 layers, 3154073 parameters, 0 gradients, 8.6 GFLOPs

PyTorch: starting from 'models/eyes_256_auto.pt' with input shape (3, 3, 256, 256) BCHW and output shape(s) (3, 35, 1344) (6.2 MB)

ONNX: starting export with onnx 1.14.1 opset 17...
ONNX: simplifying with onnxsim 0.4.36...
ONNX: export success ✅ 0.5s, saved as 'models/eyes_256_auto.onnx' (12.1 MB)

Export complete (1.8s)
Results saved to /home/tobi/Desktop/eye_tracking/models
Predict:         yolo predict task=pose model=models/eyes_256_auto.onnx imgsz=256  
Validate:        yolo val task=pose model=models/eyes_256_auto.onnx imgsz=256 data=data/yolo_pose_dataset.yaml  
Visualize:       https://netron.app
Loading models/yolov8n-face-lindevs.onnx for ONNX Runtime inference...
Loading models/eyes_256_auto.onnx for ONNX Runtime 

In [ ]:
# write_video(out, "example_video/flexpressions_cropped.mp4")

## Training

In [4]:
prepare_video_annotations_for_training(val_ratio=0.1, multiply_train_data=16)

Conversion completed.


In [ ]:
# if you don't want to track training with comet don't run this cell and set usecomet=False in the next cell
init_comet(api_key="your_api_key")

In [11]:
for image_size in [64, 128, 256, 512]: # might get stuck before starting 2nd batch, if this happens just restart this cell
    train_model(pretrained_model='models/yolov8n-pose.pt', save_name=f"training_{image_size}_autotrain", imgsz=image_size, epochs=50, batch=64, workspace="comet_workspacename", usecomet=False)

New https://pypi.org/project/ultralytics/8.2.9 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.2.8 🚀 Python-3.11.9 torch-2.1.2+cu121 CUDA:0 (NVIDIA GeForce RTX 4090, 24217MiB)
engine/trainer: task=pose, mode=train, model=YOLO(
  (model): PoseModel(
    (model): Sequential(
      (0): Conv(
        (conv): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(16, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (1): Conv(
        (conv): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (2): C2f(
        (cv1): Conv(
          (conv): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats

train: Scanning /home/tobi/ai/projects/eye_tracking/data/03_train_split.cache...
train: Caching images (1.1GB RAM): 100%|██████████| 1920/1920 [00:01<00:00, 1672
val: Scanning /home/tobi/ai/projects/eye_tracking/data/03_val_split.cache... 13 
val: Caching images (0.0GB RAM): 100%|██████████| 13/13 [00:00<00:00, 848.67it/s


Plotting labels to yolov8finetune_eyes/training_512_autotrain/labels.jpg... 
optimizer: AdamW(lr=0.001, momentum=0.937) with parameter groups 63 weight(decay=0.0), 73 weight(decay=0.0005), 72 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 512 train, 512 val
Using 8 dataloader workers
Logging results to yolov8finetune_eyes/training_512_autotrain
Starting training for 50 epochs...

      Epoch    GPU_mem   box_loss  pose_loss  kobj_loss   cls_loss   dfl_loss  Instances       Size


       1/50      6.79G      2.275      9.175      1.325      3.488      1.747   
                 Class     Images  Instances      Box(P          R      mAP50  m

                   all         13         13    0.00147     0.0769    0.00137   0.000435          0          0          0          0


...

                   all         13         13      0.993          1      0.995      0.644      0.993          1      0.995      0.892



50 epochs completed in 0.050 hours.
Optimizer stripped from yolov8finetune_eyes/training_512_autotrain/weights/last.pt, 6.6MB
Optimizer stripped from yolov8finetune_eyes/training_512_autotrain/weights/best.pt, 6.6MB

Validating yolov8finetune_eyes/training_512_autotrain/weights/best.pt...
Ultralytics YOLOv8.2.8 🚀 Python-3.11.9 torch-2.1.2+cu121 CUDA:0 (NVIDIA GeForce RTX 4090, 24217MiB)
YOLOv8n-pose summary (fused): 187 layers, 3154073 parameters, 0 gradients, 8.6 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  m


                   all         13         13      0.993          1      0.995      0.644      0.993          1      0.995      0.892
Speed: 0.0ms preprocess, 0.7ms inference, 0.0ms loss, 0.4ms postprocess per image
Results saved to yolov8finetune_eyes/training_512_autotrain


COMET ERROR: We failed to read file yolov8finetune_eyes/training_512_autotrain/F1_curve.png for uploading.
Please double-check the file path and permissions
COMET ERROR: We failed to read file yolov8finetune_eyes/training_512_autotrain/P_curve.png for uploading.
Please double-check the file path and permissions
COMET ERROR: We failed to read file yolov8finetune_eyes/training_512_autotrain/R_curve.png for uploading.
Please double-check the file path and permissions
COMET ERROR: We failed to read file yolov8finetune_eyes/training_512_autotrain/PR_curve.png for uploading.
Please double-check the file path and permissions
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     name                  : mathematical_quince_6261
COMET I